# Setup

In [1]:
# dependencies
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
import time
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
# import dataset
train = pd.read_csv("data/train.csv", index_col='id')
train.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
id,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


# EDA

In [3]:
# no null values found
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2190 entries, 0 to 2189
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2190 non-null   int64  
 1   pressure       2190 non-null   float64
 2   maxtemp        2190 non-null   float64
 3   temparature    2190 non-null   float64
 4   mintemp        2190 non-null   float64
 5   dewpoint       2190 non-null   float64
 6   humidity       2190 non-null   float64
 7   cloud          2190 non-null   float64
 8   sunshine       2190 non-null   float64
 9   winddirection  2190 non-null   float64
 10  windspeed      2190 non-null   float64
 11  rainfall       2190 non-null   int64  
dtypes: float64(10), int64(2)
memory usage: 222.4 KB


In [4]:
# labels are somewhat imbalanced
train['rainfall'].value_counts()

rainfall
1    1650
0     540
Name: count, dtype: int64

# Pre-processing

## Feature Engineering

In [5]:
# Feature engineering functions come from https://www.kaggle.com/code/lekhatopil/rainfall-prediction-eda-lightgbm-catboost#Feature-Engineering

# Create a new feature `corrected_day` to ensure each year has sequential days from 1 to 365
train['corrected_day'] = (np.arange(len(train)) % 365) + 1
train['day'] = train['corrected_day']
train.drop(columns=['corrected_day'], inplace=True)   

# Create month
def map_day_to_month(day):    
    if day <= 31:
        return 1
    elif day <= 59:
        return 2
    elif day <= 90:
        return 3
    elif day <= 120:
        return 4
    elif day <= 151:
        return 5
    elif day <= 181:
        return 6
    elif day <= 212:
        return 7
    elif day <= 243:
        return 8
    elif day <= 273:
        return 9
    elif day <= 304:
        return 10
    elif day <= 334:
        return 11
    else:
        return 12

# Mapping days to corresponding months
train['month'] = train['day'].apply(map_day_to_month)

In [6]:
# This function performs feature engineering by creating new interaction, temporal,
# and transformed features to enrich the dataset for predictive modeling.

features = ['sunshine', 'cloud', 'humidity', 'pressure', 'dewpoint', 'windspeed']  

def feature_engineering(df, features=features):
    df['cloud_humidity'] = df['cloud'] * df['humidity']
    df['pressure_humidity'] = df['pressure'] * df['humidity']
    df['sunshine_temperature'] = df['sunshine'] * df['temparature']
    df['humidity_sunshine'] = df['humidity'] * df['sunshine']
    df['cloud_wind'] = df['cloud'] * df['windspeed']
    df['inverse_humidity'] = 100 - df['humidity']
    
    # Ratio              
    df['humidity_pressure_ratio'] = df['humidity'] / df['pressure']
    df['cloud_sunshine_ratio'] = df['cloud'] / (df['sunshine'] + 1)
    df['cloud_humidity_ratio'] = df['cloud'] / (df['humidity'] + 1)

    # May detect `Rain-Bearing Winds`
    # Positve value ->> wind blows from west to east 
    # Negative value ->> wind blows from east to west 
    df['wind_west_east'] = df['windspeed'] * np.cos(np.radians(df['winddirection']))
    
    # Positive value ->> wind blows from south to north
    # Negative value ->> wind blows from north to south 
    df['wind_south_north'] = df['windspeed'] * np.sin(np.radians(df['winddirection']))

    # Wind magnitude 
    df['wind_magnitude'] = np.sqrt(df['wind_west_east']** 2 + df['wind_south_north']**2) 
    
    # Range
    df['temp_range'] = df['maxtemp'] - df['mintemp']

    # `pressure_dewpoint_diff` might help detect Fog/Mist 
    df['pressure_dewpoint_diff'] = df['pressure'] - df['dewpoint']
    
    # Might indicate how close air is to saturation
    df['temperature_dewpoint_diff'] = df['temparature'] - df['dewpoint']

    # Cycling encoding for `day` and `month`
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 365)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Relative features 
    for feature in features:
        df[f'{feature}_relative'] = df[feature] / df.groupby('month')[feature].transform('max') 
        
    # Lag features 
    for feature in features:
        for lag in [1, 2, 3]:
            df[f'{feature}_lag{lag}'] = df[feature].shift(lag).fillna(df[feature].median()) 

    # Rolling Median
    for feature in features:
        df[f'{feature}_rollings3'] = df[feature].rolling(3, min_periods=1).median()
      
    return df      

train_df = feature_engineering(train)

## Splitting

In [7]:
# separate features
X = train.drop('rainfall', axis=1)
y = train['rainfall']

# define data matrix
dmatrix = xgb.DMatrix(data=X, label=y)

X.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,...,dewpoint_lag3,windspeed_lag1,windspeed_lag2,windspeed_lag3,sunshine_rollings3,cloud_rollings3,humidity_rollings3,pressure_rollings3,dewpoint_rollings3,windspeed_rollings3
id,,,,,,,,,,,,,,,,,,,,,
0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,...,22.15,20.5,20.5,20.5,1.10,88.0,87.0,1017.40,19.4,17.20
1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,...,22.15,17.2,20.5,20.5,0.55,89.5,91.0,1018.45,17.4,19.55
2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,...,22.15,21.9,17.2,20.5,1.10,88.0,87.0,1019.50,15.4,18.10
3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,...,19.40,18.1,21.9,17.2,0.00,91.0,95.0,1019.50,15.4,21.90
4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,...,15.40,35.6,18.1,21.9,3.60,47.0,75.0,1021.80,9.6,24.80


In [8]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1738)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1752, 61) (438, 61) (1752,) (438,)


# Modeling

## Base Model

In [9]:
# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }         
           
          
# instantiate the classifier 
xgb_clf = xgb.XGBClassifier(**params)


# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1.0, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [10]:
# evaluate
y_pred = xgb_clf.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"AUC: {roc_auc_score(y_test, y_pred)}")

Accuracy: 0.863013698630137
AUC: 0.796969696969697


## Hyperparameter Tuning

In [11]:
xgb_clf = xgb.XGBClassifier(learning_rate = 0.02,
              n_estimators=100,
              max_depth=5,
              min_child_weight=2,
              gamma=0.1,
              subsample=0.85,
              colsample_bytree=0.8,
              objective = 'binary:logistic',
              nthread=4,
              scale_pos_weight=2,
              seed=1738)

def gsearch(param_test, estimator=xgb_clf, X_train=X_train, y_train=y_train):
    search = GridSearchCV(estimator=estimator, 
                param_grid = param_test,
                scoring='roc_auc',
                n_jobs=-1,
                cv=5,
                verbose=10)

    search.fit(X_train, y_train)
    display(pd.DataFrame(search.cv_results_))
    print('Best Grid Search Parameters:',search.best_params_)
    print('Best Grid Search Score: ',search.best_score_)
    return search.best_params_, search.best_score_

In [12]:
# limited space based on previous iterations
param_space = {
    'max_depth': range(2, 5),
    'min_child_weight': range(5,8),
    'gamma': [i/10.0 for i in range(3)],
    'subsample': [i/10.0 for i in range(5,8)],
    'colsample_bytree': [i/10.0 for i in range(5,8)],
    'reg_alpha': range(4, 13, 2)
}

In [13]:
# grid CV search for optimal parameters
start_time = time.time()

im_best_params, im_best_score = gsearch(param_test=param_space)

end_time = time.time()
print(f"Time elapsed: {round((end_time-start_time)/60, 2)} minutes")

Fitting 5 folds for each of 1215 candidates, totalling 6075 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_gamma,param_max_depth,param_min_child_weight,param_reg_alpha,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.268757,0.072908,0.018606,0.008495,0.5,0.0,2,5,4,0.5,"{'colsample_bytree': 0.5, 'gamma': 0.0, 'max_d...",0.879833,0.881879,0.876321,0.898322,0.884954,0.884262,0.007570,47
1,0.274927,0.117731,0.026608,0.016285,0.5,0.0,2,5,4,0.6,"{'colsample_bytree': 0.5, 'gamma': 0.0, 'max_d...",0.881879,0.885057,0.875991,0.899093,0.883721,0.885148,0.007629,6
2,0.304863,0.107422,0.018744,0.008369,0.5,0.0,2,5,4,0.7,"{'colsample_bytree': 0.5, 'gamma': 0.0, 'max_d...",0.881422,0.882619,0.872005,0.898762,0.885769,0.884116,0.008642,51
3,0.267473,0.102793,0.023607,0.010714,0.5,0.0,2,5,6,0.5,"{'colsample_bytree': 0.5, 'gamma': 0.0, 'max_d...",0.877177,0.879397,0.874516,0.897485,0.887156,0.883146,0.008317,166
4,0.253430,0.098874,0.019472,0.007646,0.5,0.0,2,5,6,0.6,"{'colsample_bytree': 0.5, 'gamma': 0.0, 'max_d...",0.879637,0.883599,0.875308,0.895701,0.883501,0.883549,0.006796,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,0.450572,0.014784,0.013838,0.000814,0.7,0.2,4,7,10,0.6,"{'colsample_bytree': 0.7, 'gamma': 0.2, 'max_d...",0.873803,0.881705,0.876542,0.886408,0.878039,0.879299,0.004375,1009
1211,0.450760,0.009966,0.014005,0.000632,0.7,0.2,4,7,10,0.7,"{'colsample_bytree': 0.7, 'gamma': 0.2, 'max_d...",0.871103,0.883882,0.871895,0.886452,0.880109,0.878688,0.006212,1093
1212,0.388082,0.015348,0.013205,0.001030,0.7,0.2,4,7,12,0.5,"{'colsample_bytree': 0.7, 'gamma': 0.2, 'max_d...",0.871517,0.879049,0.879272,0.885527,0.876542,0.878381,0.004534,1136
1213,0.301558,0.026852,0.014005,0.000316,0.7,0.2,4,7,12,0.6,"{'colsample_bytree': 0.7, 'gamma': 0.2, 'max_d...",0.873084,0.882140,0.875529,0.886187,0.874163,0.878221,0.005078,1148


Best Grid Search Parameters: {'colsample_bytree': 0.7, 'gamma': 0.2, 'max_depth': 2, 'min_child_weight': 7, 'reg_alpha': 4, 'subsample': 0.6}
Best Grid Search Score:  0.8853089839858084
Time elapsed: 1.71 minutes


In [14]:
# search for learning rate, n_estimators, and scale_pos_weight

# create model with intermediate best parameters
base_params = {
            'objective':'binary:logistic',
            'max_depth': 3,
            'alpha': 10,
            'learning_rate': 0.1,
            'n_estimators': 100,
            'min_child_weight': 1,
            'gamma': 0.1,
            'subsample': 0.9,
            'colsample_bytree': 0.7,
            'scale_pos_weight': 2,
            'seed': 1738,
            'eval_metric': 'auc'
        }

im_params = base_params | im_best_params

# define the intermediate estimator 
xgb_clf_im = xgb.XGBClassifier(**im_params)

# search for new parameters
im_param_space = {
    'learning_rate': np.arange(0.005, 0.051, 0.005),
    'n_estimators': range(10, 300, 20)
}

start_time = time.time()

best_params, best_score = gsearch(param_test=im_param_space)

end_time = time.time()
print(f"Time elapsed: {round((end_time-start_time)/60, 2)} minutes")

Fitting 5 folds for each of 150 candidates, totalling 750 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.137618,0.049746,0.012926,0.000537,0.005,10,"{'learning_rate': 0.005, 'n_estimators': 10}",0.868883,0.879463,0.864231,0.867777,0.882223,0.872515,0.007026,148
1,0.281143,0.100103,0.013706,0.001167,0.005,30,"{'learning_rate': 0.005, 'n_estimators': 30}",0.870363,0.884274,0.866279,0.884338,0.888566,0.878764,0.008763,69
2,0.463083,0.020811,0.014708,0.001250,0.005,50,"{'learning_rate': 0.005, 'n_estimators': 50}",0.867816,0.883925,0.865486,0.889072,0.891297,0.879519,0.010801,47
3,0.599743,0.012174,0.014306,0.001209,0.005,70,"{'learning_rate': 0.005, 'n_estimators': 70}",0.868774,0.883447,0.866081,0.891010,0.890284,0.879919,0.010570,23
4,0.747570,0.029528,0.013986,0.000679,0.005,90,"{'learning_rate': 0.005, 'n_estimators': 90}",0.868861,0.882467,0.867292,0.890306,0.889491,0.879683,0.009874,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1.301407,0.051172,0.020807,0.013608,0.050,210,"{'learning_rate': 0.049999999999999996, 'n_est...",0.874129,0.867816,0.856457,0.886320,0.885571,0.874059,0.011240,135
146,1.360925,0.047927,0.013704,0.000677,0.050,230,"{'learning_rate': 0.049999999999999996, 'n_est...",0.872692,0.868513,0.855136,0.885923,0.885218,0.873496,0.011441,137
147,1.353705,0.040007,0.013007,0.000450,0.050,250,"{'learning_rate': 0.049999999999999996, 'n_est...",0.871648,0.865552,0.854299,0.886320,0.885439,0.872651,0.012154,146
148,1.213154,0.073762,0.014004,0.001001,0.050,270,"{'learning_rate': 0.049999999999999996, 'n_est...",0.871125,0.864899,0.854431,0.887112,0.884338,0.872381,0.012163,149


Best Grid Search Parameters: {'learning_rate': 0.025, 'n_estimators': 70}
Best Grid Search Score:  0.8811350270953318
Time elapsed: 0.55 minutes


In [15]:
# create model with best parameters
tuned_params = im_params | best_params

# instantiate the classifier 
xgb_clf_hp = xgb.XGBClassifier(**tuned_params)

# fit the classifier to the training data
xgb_clf_hp.fit(X_train, y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=0.2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.025, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=70, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
# evaluate train
y_train_prob = [i[1] for i in xgb_clf_hp.predict_proba(X_train)]
y_train_pred = xgb_clf_hp.predict(X_train)

print(f"Train Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"Train Pred AUC: {roc_auc_score(y_train, y_train_pred)}")
print(f"Train Prob AUC: {roc_auc_score(y_train, y_train_prob)}")

Train Accuracy: 0.8515981735159818
Train Pred AUC: 0.7193181818181819
Train Prob AUC: 0.9052890011223343


In [17]:
# evaluate test
y_test_prob = [i[1] for i in xgb_clf_hp.predict_proba(X_test)]
y_test_pred = xgb_clf_hp.predict(X_test)

print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(f"Test Pred AUC: {roc_auc_score(y_test, y_test_pred)}")
print(f"Test Prob AUC: {roc_auc_score(y_test, y_test_prob)}")

Test Accuracy: 0.8333333333333334
Test Pred AUC: 0.69006734006734
Test Prob AUC: 0.9004208754208755


## Re-train with all data

Comparable training and testing performance suggests model is not over-fitting

Will re-train the model on all available training data, then re-evaluate with cross-validation

In [23]:
# fit the classifier to the training data
xgb_clf_final = xgb.XGBClassifier(**tuned_params)
xgb_clf_final.fit(X, y)

# cross-validation scoring
print(f"Average Accuracy: {np.mean(cross_val_score(xgb_clf_final, X, y, cv=5, scoring='accuracy'))}")
print(f"Average AUC: {np.mean(cross_val_score(xgb_clf_final, X, y, cv=5, scoring='roc_auc'))}")

Average Accuracy: 0.838812785388128
Average AUC: 0.887384960718294


# Submission

In [19]:
# import test data
test = pd.read_csv("data/test.csv", index_col='id')
test.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
id,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,24.3
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,35.3
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,16.9
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,50.6
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,19.4


In [20]:
# apply pre-processing to test data
test['month'] = test['day'].apply(map_day_to_month)
test_df = feature_engineering(test)
test_df.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,...,dewpoint_lag3,windspeed_lag1,windspeed_lag2,windspeed_lag3,sunshine_rollings3,cloud_rollings3,humidity_rollings3,pressure_rollings3,dewpoint_rollings3,windspeed_rollings3
id,,,,,,,,,,,,,,,,,,,,,
2190,1,1019.5,17.5,15.8,12.7,14.9,96.0,99.0,0.0,50.0,...,22.3,21.3,21.3,21.3,0.0,99.0,96.0,1019.5,14.9,24.3
2191,2,1016.5,17.5,16.5,15.8,15.1,97.0,99.0,0.0,50.0,...,22.3,24.3,21.3,21.3,0.0,99.0,96.5,1018.0,15.0,29.8
2192,3,1023.9,11.2,10.4,9.4,8.9,86.0,96.0,0.0,40.0,...,22.3,35.3,24.3,21.3,0.0,99.0,96.0,1019.5,14.9,24.3
2193,4,1022.9,20.6,17.3,15.2,9.5,75.0,45.0,7.1,20.0,...,14.9,16.9,35.3,24.3,0.0,96.0,86.0,1022.9,9.5,35.3
2194,5,1022.2,16.1,13.8,6.4,4.3,68.0,49.0,9.2,20.0,...,15.1,50.6,16.9,35.3,7.1,49.0,75.0,1022.9,8.9,19.4


In [21]:
# read in sample submission format
sample_submission = pd.read_csv("data/sample_submission.csv")
sample_submission

,id,rainfall
0,2190,0
1,2191,0
2,2192,0
3,2193,0
4,2194,0
...,...,...
725,2915,0
726,2916,0
727,2917,0
728,2918,0


In [22]:
# save submissions and export
preds = [i[1] for i in xgb_clf_final.predict_proba(test)]
sample_submission['rainfall'] = preds
display(sample_submission.head())
sample_submission.to_csv("submissions/rainfall_submission.csv", index=False)

,id,rainfall
0,2190,0.943455
1,2191,0.943455
2,2192,0.937593
3,2193,0.384963
4,2194,0.312128
